In [ ]:
import re
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv; load_dotenv()
from tqdm import tqdm

In [ ]:
df = pd.read_csv("data/urls-20240616.csv")
df["text"] = df.subject + " " + df.body
texts = df.text.to_list()

In [ ]:
def extract_urls(text):
    url_pattern = re.compile(r'(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+')
    urls = re.findall(url_pattern, text)
    for i, url in enumerate(urls):
        if not url.startswith(("http://", "https://")):
            urls[i] = "http://" + url
    return urls

In [ ]:
results = []

for text in texts:
    try:
        urls = extract_urls(text)
        results.extend(urls)
    except Exception as ex:
        print(text, ex)

In [ ]:
final = pd.DataFrame({ "url": results })
final = final.drop_duplicates().dropna()
final.to_csv("data/urls-processed-20240616.csv", index=False)
print(f"There are {len(final)} URLs to index.")

In [11]:
admin_api_key = os.getenv("ADMIN_API_KEY")
user_id = "e3158e77-161e-478d-89c7-8b2ef9c85474"
headers = {
    "x-admin-api-key": admin_api_key
}

for url in tqdm(results, desc="Indexing URLs..."):
    try:
        requests.post("http://localhost:8000/api/admin/index", headers=headers, json={"urls": [url]}, params={"user_id": user_id})
        time.sleep(2)
    except Exception as ex:
        print(ex)

Indexing URLs...:  67%|██████▋   | 1846/2767 [2:50:23<1:25:00,  5.54s/it]  


KeyboardInterrupt: 